In [ ]:
from google.colab import drive
# Google Drive mounten
drive.mount('/content/drive')

Mounted at /content/drive


# Topic Modeling: Gruppen und Kanäle der Influencer

In [ ]:
import pandas as pd
import re

# Lade den CSV-Datensatz
df = pd.read_csv(f'/content/drive/My Drive/MasterArbeitDaten/alle_transkripteInfluencer.csv')

# Definiere die Bedingungen für inhaltsleere Transkripte
# Prüfen auf leere Zeilen, "...", "Musik", "SWR", "Untertitel", und auf wiederholende/inhaltsschwache Transkripte
inhaltsleere_bedingungen = df['Inhalt'].isnull() | \
                           df['Inhalt'].str.strip().isin(['', '...', 'Musik']) | \
                           df['Inhalt'].str.contains(r'\bSWR\b', flags=re.IGNORECASE, na=False) | \
                           df['Inhalt'].str.contains(r'^Musik\s*(\w+\s*){0,10}', flags=re.IGNORECASE, na=False) | \
                           df['Inhalt'].str.contains(r'(\b\w+\b\s*)\1{5,}', na=False) | \
                           df['Inhalt'].str.contains(r'\bUntertitel(ung)?\b\s*(\w+\s*){0,10}', flags=re.IGNORECASE, na=False)

# Entferne die inhaltsleeren Transkripte
df_bereinigt = df[~inhaltsleere_bedingungen]

# Funktion zur Identifizierung von Transkripten mit Wortwiederholungen
def has_repetitive_text(text):
    # Prüfen auf einfache Wiederholungen wie "oh, oh, oh" oder "geil, geil"
    if re.search(r'(\b\w+\b)(\s*\1){3,}', text):
        return True
    # Prüfen auf komplexere Wiederholungen wie "Ren! Bitte, Stimpy! Ren! Bitte, Stimpy!"
    if re.search(r'((\b\w+\b\s*){1,3})(\s*\1){3,}', text):
        return True
    return False

# Filtere die Transkripte, die Wortwiederholungen enthalten
df_bereinigt = df_bereinigt[~df_bereinigt['Inhalt'].apply(has_repetitive_text)]

# Speichere den bereinigten Datensatz in einer neuen CSV-Datei
df_bereinigt.to_csv("bereinigte_transkripte.csv", index=False)

print(f"{len(df) - len(df_bereinigt)} inhaltsleere oder wiederholende Transkripte wurden entfernt.")


<ipython-input-2-83d366a7d7e9>:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['Inhalt'].str.contains(r'^Musik\s*(\w+\s*){0,10}', flags=re.IGNORECASE, na=False) | \
<ipython-input-2-83d366a7d7e9>:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['Inhalt'].str.contains(r'(\b\w+\b\s*)\1{5,}', na=False) | \
<ipython-input-2-83d366a7d7e9>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['Inhalt'].str.contains(r'\bUntertitel(ung)?\b\s*(\w+\s*){0,10}', flags=re.IGNORECASE, na=False)


2178 inhaltsleere oder wiederholende Transkripte wurden entfernt.


In [ ]:
!pip install sentence-transformers
!pip install -q bertopic
import nltk
import pandas as pd
from nltk.corpus import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer


# Falls nötig, lade die Stopwörter (einmalig ausführen)
nltk.download('stopwords')

# Lade den bereinigten CSV-Datensatz
df_bereinigt = pd.read_csv("bereinigte_transkripte_Influencer.csv")

# Definiere die deutsche Stopwortliste (du kannst auch andere Sprachen wählen)
stop_words_de = set(stopwords.words('german'))
stop_words_en = set(stopwords.words('english'))

# Kombiniere beide Stopwortlisten
combined_stop_words = stop_words_de.union(stop_words_en)

# Funktion zur Entfernung von Stopwörtern aus einem Transkript
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in combined_stop_words]
    return ' '.join(filtered_words)

# Entferne die Stopwörter aus allen Transkripten
df_bereinigt['Inhalt'] = df_bereinigt['Inhalt'].apply(remove_stopwords)

# Extrahiere die Transkripte (angenommen, die Spalte heißt 'transkript')
transkripte = df_bereinigt['Inhalt'].tolist()

# Lade ein vortrainiertes Embedding-Modell
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Erstelle das BERTopic-Modell mit Embeddings
topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True,embedding_model=embedding_model, min_topic_size=30)

# Führe das Topic Modeling durch
topics, probabilities = topic_model.fit_transform(transkripte)

# Speichere das BERTopic-Modell für die zukünftige Verwendung
topic_model.save("bertopic_influencer_transkripte")

# Zeige die 20 häufigsten Themen an
topic_info = topic_model.get_topic_info()

# Speichere die Themeninformationen in einer CSV-Datei
topic_info = topic_model.get_topic_info()
topic_info.to_csv("themen_info_transkripte_influencer.csv", index=False)

print("Die Ergebnisse wurden in 'themen_info.csv' gespeichert.")

top_20_themen = topic_info.head(21)
print(top_20_themen)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.3 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2024-10-23 10:41:32,810 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/327 [00:00<?, ?it/s]

2024-10-23 10:57:44,866 - BERTopic - Embedding - Completed ✓
2024-10-23 10:57:44,869 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-10-23 10:58:31,008 - BERTopic - Dimensionality - Completed ✓
2024-10-23 10:58:31,010 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-10-23 10:58:33,752 - BERTopic - Cluster - Completed ✓
2024-10-23 10:58:33,781 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-10-23 10:58:36,241 - BERTopic - Representation - Completed ✓
2024-10-23 10:58:36,995 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


Die Ergebnisse wurden in 'themen_info.csv' gespeichert.
    Topic  Count                                     Name  \
0      -1   4333                    -1_oh_ja_mal_freiheit   
1       0   1705                0_impfung_ist_menschen_ja   
2       1    474           1_abstand_halten_ruhe_schwarze   
3       2    399        2_polizei_alleine_verp_feindliche   
4       3    371               3_tommy_stopp_mensch_lügen   
5       4    254              4_trump_die_biden_präsident   
6       5    246  5_berlin_nein_demonstrieren_deutschland   
7       6    233                     6_co2_euro_mehr_geld   
8       7    197                 7_geh_au_haben_passieren   
9       8    187              8_nazi_nein_polizei_gesehen   
10      9    174                  9_dr_polizei_hey_runter   
11     10    167               10_german_landes_volk_nähe   
12     11    123                11_leute_wien_heute_grüße   
13     12    118              12_stadt_kalt_mitte_kulisse   
14     13    110             

# Topic Modeling: Lokal Gruppen und Kanäle

In [ ]:
import pandas as pd
import re

# Lade den CSV-Datensatz
df = pd.read_csv(f'/content/drive/My Drive/MasterArbeitDatenLokal/LokaleGruppen/alle_transkripteLokal.csv')

# Definiere die Bedingungen für inhaltsleere Transkripte
inhaltsleere_bedingungen = df['Inhalt'].isnull() | \
                           df['Inhalt'].str.strip().isin(['', '...', 'Musik']) | \
                           df['Inhalt'].str.contains(r'\bSWR\b', flags=re.IGNORECASE, na=False) | \
                           df['Inhalt'].str.contains(r'^Musik\s*(\w+\s*){0,10}', flags=re.IGNORECASE, na=False) | \
                           df['Inhalt'].str.contains(r'(\b\w+\b\s*)\1{5,}', na=False) | \
                           df['Inhalt'].str.contains(r'\bUntertitel(ung)?\b\s*(\w+\s*){0,10}', flags=re.IGNORECASE, na=False)

# Entferne die inhaltsleeren Transkripte
df_bereinigt = df[~inhaltsleere_bedingungen]

# Funktion zur Identifizierung von Transkripten mit Wortwiederholungen
def has_repetitive_text(text):
    # Prüfen auf einfache Wiederholungen wie "oh, oh, oh" oder "geil, geil"
    if re.search(r'(\b\w+\b)(\s*\1){3,}', text):
        return True
    # Prüfen auf komplexere Wiederholungen wie "Ren! Bitte, Stimpy! Ren! Bitte, Stimpy!"
    if re.search(r'((\b\w+\b\s*){1,3})(\s*\1){3,}', text):
        return True
    return False

# Filtere die Transkripte, die Wortwiederholungen enthalten
df_bereinigt = df_bereinigt[~df_bereinigt['Inhalt'].apply(has_repetitive_text)]

# Speichere den bereinigten Datensatz in einer neuen CSV-Datei
df_bereinigt.to_csv("bereinigte_transkripte_Lokal.csv", index=False)

print(f"{len(df) - len(df_bereinigt)} Transkripte mit Wortwiederholungen wurden entfernt.")




2774 Transkripte mit Wortwiederholungen wurden entfernt.


Verändern der min_topic_size für eine grobere Zusammenfassung um Rauschen in den Detais zu umgehen.

In [ ]:
!pip install sentence-transformers
!pip install -q bertopic
import nltk
import pandas as pd
from nltk.corpus import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer


# Falls nötig, lade die Stopwörter (einmalig ausführen)
nltk.download('stopwords')

# Lade den bereinigten CSV-Datensatz
df_bereinigt = pd.read_csv("bereinigte_transkripte_Lokal.csv")

# Definiere die deutsche Stopwortliste (du kannst auch andere Sprachen wählen)
stop_words_de = set(stopwords.words('german'))
stop_words_en = set(stopwords.words('english'))

# Kombiniere beide Stopwortlisten
combined_stop_words = stop_words_de.union(stop_words_en)

# Funktion zur Entfernung von Stopwörtern aus einem Transkript
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in combined_stop_words]
    return ' '.join(filtered_words)

# Entferne die Stopwörter aus allen Transkripten
df_bereinigt['Inhalt'] = df_bereinigt['Inhalt'].apply(remove_stopwords)

# Extrahiere die Transkripte (angenommen, die Spalte heißt 'transkript')
transkripte = df_bereinigt['Inhalt'].tolist()

# Lade ein vortrainiertes Embedding-Modell
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Erstelle das BERTopic-Modell mit Embeddings
topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True,embedding_model=embedding_model, min_topic_size=30)

# Führe das Topic Modeling durch
topics, probabilities = topic_model.fit_transform(transkripte)

# Speichere das BERTopic-Modell für die zukünftige Verwendung
topic_model.save("bertopic_lokal_transkripte")

# Zeige die 20 häufigsten Themen an
topic_info = topic_model.get_topic_info()

# Speichere die Themeninformationen in einer CSV-Datei
topic_info = topic_model.get_topic_info()
topic_info.to_csv("themen_info_transkripte_lokal_30.csv", index=False)

print("Die Ergebnisse wurden in 'themen_info.csv' gespeichert.")

top_20_themen = topic_info.head(21)
print(top_20_themen)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2024-10-23 15:54:47,345 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/511 [00:00<?, ?it/s]

2024-10-23 16:21:47,594 - BERTopic - Embedding - Completed ✓
2024-10-23 16:21:47,596 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-10-23 16:22:14,707 - BERTopic - Dimensionality - Completed ✓
2024-10-23 16:22:14,710 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-10-23 16:22:26,622 - BERTopic - Cluster - Completed ✓
2024-10-23 16:22:26,636 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-10-23 16:22:30,263 - BERTopic - Representation - Completed ✓
2024-10-23 16:22:31,717 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


Die Ergebnisse wurden in 'themen_info.csv' gespeichert.
    Topic  Count                                          Name  \
0      -1   7971                 -1_ist_freiheit_mehr_menschen   
1       0   1005                         0_komm_scheiße_oh_wow   
2       1    442         1_kanada_impfung_impfstoff_aufschauen   
3       2    333                     2_polizei_verp_nein_durch   
4       3    243               3_2019_problem_schleimhaut_euro   
5       4    240                           4_ging_nicht_000_ja   
6       5    186                      5_spread_hei_stadt_hölle   
7       6    184         6_osten_vielen_politik_entscheidungen   
8       7    181                      7_dr_monday_runter_wohin   
9       8    176                      8_liebe_leben_medien_usa   
10      9    170                      9_euch_schämt_nein_maske   
11     10    166                 10_kinder_maske_schule_tragen   
12     11    161                          11_pcr_tests_test_ct   
13     12    152  12

Anmerkung: min topic size 100 ist viel zu groß; bei Topic Size 60 gibt es nur 9 Themen. Letzter Versuch ist 55 ergibt nur drei Topics. Schlussfolgerung: 50 scheint am Besten zu sein

In [ ]:
!pip install sentence-transformers
!pip install -q bertopic
import nltk
import pandas as pd
from nltk.corpus import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer


# Falls nötig, lade die Stopwörter (einmalig ausführen)
nltk.download('stopwords')

# Lade den bereinigten CSV-Datensatz
df_bereinigt = pd.read_csv("bereinigte_transkripte_Lokal.csv")

# Definiere die deutsche Stopwortliste (du kannst auch andere Sprachen wählen)
stop_words_de = set(stopwords.words('german'))
stop_words_en = set(stopwords.words('english'))

# Kombiniere beide Stopwortlisten
combined_stop_words = stop_words_de.union(stop_words_en)

# Funktion zur Entfernung von Stopwörtern aus einem Transkript
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in combined_stop_words]
    return ' '.join(filtered_words)

# Entferne die Stopwörter aus allen Transkripten
df_bereinigt['Inhalt'] = df_bereinigt['Inhalt'].apply(remove_stopwords)

# Extrahiere die Transkripte (angenommen, die Spalte heißt 'transkript')
transkripte = df_bereinigt['Inhalt'].tolist()

# Lade ein vortrainiertes Embedding-Modell
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Erstelle das BERTopic-Modell mit Embeddings
topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True,embedding_model=embedding_model,min_topic_size=50)

# Führe das Topic Modeling durch
topics, probabilities = topic_model.fit_transform(transkripte)

# Speichere das BERTopic-Modell für die zukünftige Verwendung
topic_model.save("bertopic_lokal_transkripte")

# Zeige die 20 häufigsten Themen an
topic_info = topic_model.get_topic_info()

# Speichere die Themeninformationen in einer CSV-Datei
topic_info = topic_model.get_topic_info()
topic_info.to_csv("themen_info_transkripte_lokal.csv", index=False)

print("Die Ergebnisse wurden in 'themen_info.csv' gespeichert.")

top_20_themen = topic_info.head(21)
print(top_20_themen)
